In [1]:
#!/usr/bin/env python
 
'''
Selfies.ipynb: A notebook to align a process DSLR selfies from our adventures
A. J. McCulloch, March 2020
'''

%matplotlib notebook

####################################################################################################
# Import modules
####################################################################################################

import glob
import os
import csv
import math
from PIL import Image
from PIL.ExifTags import TAGS

####################################################################################################
# Define functions
####################################################################################################

# get the exif info from the JPEG files
def get_exif(fn):
    ret = {}
    i = Image.open(fn)
    info = i._getexif()
    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        ret[decoded] = value
    return ret

# get the gps coordinates from the exif
def getgps(img):
    gps=None
    fulldata=get_exif(img)
    if 'GPSInfo' in fulldata:
        gps=fulldata['GPSInfo']
    return gps

# get the degrees, minutes and seconds from the GPS value
def getdms(dat):
    d = int(dat[0][0])
    m = int(float(dat[1][0])/float(dat[1][1]))
    s = (float(dat[1][0])/float(dat[1][1]) - m) * 60    
    return [d,m,s]

# rentun the gps value in dms with "N,S,E,W"    
def interpretgpsdms(coords):
    lat = []
    long = []
    
    lat = getdms(coords[2])
    long = getdms(coords[4])
    lat.append(coords[1])
    long.append(coords[3])
    return [lat,long]
    
def dmstodec(dat):
    dat[0]+(dat[1]/60)

def interpretgpsdec(coords):
    lat=x[0][0]+(x[0][1]+(x[0][2]/60))/60
    if x[0][3] != 'N':
        lat = 0 - lat 
    
    long=x[1][0]+(x[1][1]+(x[1][2]/60))/60
    if x[1][3] != 'E':
        long = 0 - long
    
    return [lat,long]

# find the images from which to extract metadata
images=[]
image_files = glob.glob('Images/*.jpg')

# get the gps values for all the images
data=[]
for image in image_files:        
    data.append(getgps(image))

data=filter(None,data)
fdata=[i for i in data]
    
# interpret the gps values of the images    
intdata=[]
for x in fdata:
    intdata.append(interpretgpsdms(x))
    
decdata = []
for x in intdata:
    decdata.append(interpretgpsdec(x))


In [2]:
# Construct a list of longitude, latitude values required for cartopy plotting
dd = [] # initialise the array

for i in decdata:
    dd.append([[i[1]], [i[0]]])
dd.reverse()

In [7]:
####################################################################################################
# Import modules
####################################################################################################

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from pylab import *

# Set the central longitude of the map: 10 does not split continents
central_longitude = 10

# Initialise the figure
fig = plt.figure(figsize=(19.2,10.8), dpi = 100)

# Use cartopy to crease a map located at the central longitude
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
# Set a custom background for the map (NASA blue marble, from https://pyresample.readthedocs.io/en/latest/plot.html)
ax.background_img(name='BM')
# Crop the map slightly
ax.set_extent([-180, 179, -75, 75], crs=ccrs.PlateCarree())

def update(frame_number):
    for d in dd[:frame_number]:
        plt.plot(d[0],d[1], marker='o', color = '#ecc81a', markersize=2, alpha=0.8, linestyle='', transform=ccrs.PlateCarree())

for frame in range(len(dd)):
    update(frame)
    plt.savefig('Output/frame'+str(frame)+'.png')
    
# Construct the animation, using the update function as the animation director.
#ani = animation.FuncAnimation(fig, update, interval=10, save_count=len(dd))

#ani.save("movie.mp4")

#plt.show()

<IPython.core.display.Javascript object>

Exception ignored in: <function Comm.__del__ at 0x0000022AA29C7558>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\comm\comm.py", line 76, in __del__
    self.close()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\comm\comm.py", line 116, in close
    self.kernel.comm_manager.unregister_comm(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\comm\manager.py", line 56, in unregister_comm
    comm = self.comms.pop(comm.comm_id)
KeyError: ('ed7884c169d5424e9d97f1e345364cec',)
Exception ignored in: <function Comm.__del__ at 0x0000022AA29C7558>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\comm\comm.py", line 76, in __del__
    self.close()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\comm\comm.py", line 116, in close
    self.kernel.comm_manager.unregister_comm(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\comm\manager.py", li

KeyboardInterrupt: 

In [5]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib notebook

def data_gen():
    for cnt in range(1000):
        t = cnt / 10
        yield t, np.sin(2*np.pi*t) * np.exp(-t/10.)


def init():
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlim(0, 10)
    del xdata[:]
    del ydata[:]
    line.set_data(xdata, ydata)
    return line,

fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)
ax.grid()
xdata, ydata = [], []


def run(data):
    # update the data
    t, y = data
    xdata.append(t)
    ydata.append(y)
    xmin, xmax = ax.get_xlim()

    if t >= xmax:
        ax.set_xlim(xmin, 2*xmax)
        ax.figure.canvas.draw()
    line.set_data(xdata, ydata)

    return line,

ani = animation.FuncAnimation(fig, run, data_gen, blit=False, interval=10,
                              repeat=False, init_func=init)
plt.show()

<IPython.core.display.Javascript object>

In [4]:
%matplotlib widget

In [ ]:
def init():
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
    ax.stock_img()
    
    return ax,

fig, ax = plt.subplots(figsize=(14,9), dpi = 300)

ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
ax.stock_img()

def animate(i):
    for d in dd[:i]:
        plt.plot(d[0],d[1], marker='o', color = 'C11', markersize=1, linestyle='', transform=ccrs.PlateCarree())
    
    

In [ ]:
ani = animation.FuncAnimation(
    fig, animate, init_func=init, interval=1, blit=True, save_count=50)

In [ ]:
for i in []:
    print('o')

In [ ]:
dd = []

for i in decdata:
    dd.append([[i[1]], [i[0]]])
dd.reverse()
    

In [ ]:
dd[0]

# Image alignment

In [ ]:
# import the necessary packages
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import argparse
import imutils
import dlib
import cv2

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True,
help="path to facial landmark predictor")
ap.add_argument("-i", "--image", required=True,
help="path to input image")
args = vars(ap.parse_args())

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor and the face aligner
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])
fa = FaceAligner(predictor, desiredFaceWidth=256)

In [ ]:
# load the input image, resize it, and convert it to grayscale
image = cv2.imread(args["image"])
image = imutils.resize(image, width=800)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# show the original input image and detect faces in the grayscale
# image
cv2.imshow("Input", image)
rects = detector(gray, 2)

In [ ]:
# loop over the face detections
for rect in rects:
    # extract the ROI of the *original* face, then align the face
    # using facial landmarks
    (x, y, w, h) = rect_to_bb(rect)
    faceOrig = imutils.resize(image[y:y + h, x:x + w], width=256)
    faceAligned = fa.align(image, gray, rect)
    # display the output images
    cv2.imshow("Original", faceOrig)
    cv2.imshow("Aligned", faceAligned)
    cv2.waitKey(0)

In [ ]:
image_files